# Download and prepare the data

## Project description

Beta Bank customers are leaving: little by little, chipping away every month. The bankers figured out it’s cheaper to save the existing customers rather than to attract new ones.

We need to predict whether a customer will leave the bank soon. We have the data on clients’ past behavior and termination of contracts with the bank.

We need to Build a model with the maximum possible F1 score. The F1 score should be of at least 0.59. 
Additionally, we will measure the AUC-ROC metric and compare it with the F1.


## Import

In [102]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer


# temp import
#from pandas_profiling import ProfileReport

profile = ProfileReport(data)
profile
profile.to_file(output_file="report.html")

## Load data


In [103]:
try:
    data = pd.read_csv('Churn.csv')
except:
    data = pd.read_csv('datasets/Churn.csv')

## Check the data

**Features**

- RowNumber — data string index
- CustomerId — unique customer identifier
- Surname — surname
- CreditScore — credit score
- Geography — country of residence
- Gender — gender
- Age — age
- Tenure — period of maturation for a customer’s fixed deposit (years)
- Balance — account balance
- NumOfProducts — number of banking products used by the customer
- HasCrCard — customer has a credit card
- IsActiveMember — customer’s activeness
- EstimatedSalary — estimated salary


**Target**

- Exited — сustomer has left

In [104]:
data.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4643,4644,15672594,Stevenson,597,France,Female,60,0.0,131778.08,1,0,0,10703.53,1
8782,8783,15809057,Lu,600,Spain,Female,27,6.0,0.00,2,1,1,172031.22,0
9437,9438,15804864,Chu,670,France,Female,27,5.0,79336.61,1,1,1,26170.08,0
455,456,15733797,Sal,506,France,Male,36,5.0,0.00,2,1,0,164253.35,0
3885,3886,15618255,Fedorov,642,Germany,Female,56,6.0,103244.86,2,1,0,143049.72,1


In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Only Tenure have missing

In [106]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


CreditScore mean and median are almost identical with low std

Age mean and median are almost identical. The youngest user is 18.

Tenure mean and median are almost identical std is high. almost the same as the mean value.

Balance mean is much lower then the median. Since the Q1 is 0 we can assume lot of observations have 0 balance.

NumOfProducts varies between 1 to 4.

HasCrCard is boolean - 0 for no 1 for yes. Same for IsActiveMember.

EstimatedSalary mean and median are almost identical. High std

Exited is boolean - 0 for no 1 for yes.



## Treating missing values

We will replace the missing values with -1 to distinguish the observation with 0 deposit to places with no tracked deposit at all that also may be 0 deposit
imp

In [107]:
data['Tenure'].fillna(-1, inplace=True)

## Transforming categorical features into numerical features

In [108]:
# create dummies for Geography
data_ohe = pd.get_dummies(data['Geography'], drop_first=True)

In [109]:
# join the data with the ohe data of the Geography
data_modified = data.drop('Geography', axis=1).join(data_ohe)

In [110]:
data_modified.sample(20)

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain
2420,2421,15783305,Franklin,593,Female,46,7.0,98752.51,1,1,0,145560.38,0,0,0
5421,5422,15757661,Trevisano,589,Female,39,7.0,0.00,2,0,0,95985.64,0,0,0
5231,5232,15749519,Lin,822,Male,38,6.0,128289.70,3,1,0,9149.96,1,0,0
1509,1510,15638149,Rowley,528,Male,37,6.0,103772.45,1,1,0,197111.99,0,0,0
9476,9477,15571529,Kirby,650,Female,48,7.0,138232.24,1,1,0,57594.78,0,1,0
3102,3103,15660866,Chimaobim,640,Female,29,3.0,0.00,2,1,0,2743.69,0,0,0
3168,3169,15680752,Horrocks,675,Female,49,0.0,0.00,1,1,1,80496.71,1,0,0
9755,9756,15804009,Amechi,806,Male,36,8.0,167983.17,2,1,1,106714.28,0,1,0
3854,3855,15580935,Okechukwu,687,Male,33,-1.0,135962.40,2,1,0,121747.96,0,1,0
2292,2293,15779064,Chidiegwu,677,Male,27,2.0,0.00,2,1,1,20092.89,0,0,0


## Features selection
In the data we have RowNumber, CustomerId. This info is irrelevant for our prediction because ID are given randomly to users and row number is arbitrary. Lets see if surname is also irrelevant 

In [114]:
data_modified['Surname'].value_counts()

Smith       32
Scott       29
Martin      29
Walker      28
Brown       26
            ..
Izmailov     1
Bold         1
Bonham       1
Poninski     1
Burbidge     1
Name: Surname, Length: 2932, dtype: int64

We can also remove this column. It's irrelevant feature for us also. 

In [115]:
# remove irrelevant columns and remain only with relevant features and target
data_modified = data_modified.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1)
data_modified.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain
0,619,Female,42,2.0,0.00,1,1,1,101348.88,1,0,0
1,608,Female,41,1.0,83807.86,1,0,1,112542.58,0,0,1
2,502,Female,42,8.0,159660.80,3,1,0,113931.57,1,0,0
3,699,Female,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,Female,43,2.0,125510.82,1,1,1,79084.10,0,0,1
